# Google Playstore 

## Authenticate and create PyDrive client.

You will be prompted with a link to click on, and give permission to Google Colab to access your Google Drive. If you don't want to give permission to your personal google drive, create a new gmail account, and complete this process using the new account.

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html


## Import data
Locate the "Data" folder in your drive. Right click and click "share" to get the ID of the folder. Replace < Data folder id > with the id you got. (id should look like "1j8oG_vCmum965Ghg8LdbSkfj-lfi-AZ0" )

In [0]:
import pandas as pd
import numpy as np
import io
from time import ctime
import csv

In [3]:

file_list = drive.ListFile({'q': "'1j8oG_vCmum9YuVWP8LdbSkfj-lfi-AZ0' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: training_desc.csv, id: 1kSoeoaK_TsTHnhSGPXOLNy43jd20G8bH
title: training_data.csv, id: 12-B9GpferXatOB4ACo7GanQ2iBt5O-66
title: test_data.csv, id: 1y_YySFyibYYwQtJr6v7Qf81FqULwK7Fk
title: training_labels.csv, id: 1cqErOBjBB91P_xrt4TKuZ91OfTLoUYmx


### Pulling data into Google Colab.

In [9]:
print ("loading time starts at " ,ctime())
training_data_downloaded = drive.CreateFile({'id': '12-B9GpferXatOB4ACo7GanQ2iBt5O-66'})
training_data_downloaded.GetContentFile('training_data.csv')

training_desc_downloaded = drive.CreateFile({'id': '1kSoeoaK_TsTHnhSGPXOLNy43jd20G8bH'})
training_desc_downloaded.GetContentFile('training_desc.csv')

training_labels_downloaded = drive.CreateFile({'id': '1cqErOBjBB91P_xrt4TKuZ91OfTLoUYmx'})
training_labels_downloaded.GetContentFile('training_labels.csv')

test_data_downloaded = drive.CreateFile({'id': '1y_YySFyibYYwQtJr6v7Qf81FqULwK7Fk'})
test_data_downloaded.GetContentFile('test_data.csv') 

print ("Loading ends at " ,ctime())

('loading time starts at ', 'Sun Jun  3 08:56:09 2018')
('Loading ends at ', 'Sun Jun  3 08:56:21 2018')


###Loading Time = ~10 seconds



---


## A. Introduction (max 500 words)


The google play store provides a platform for app developers to sell their product to customers. However, with over two million apps in the store, it can be difficult for customers to find the types of apps they're looking for (Schoon, 2016). To simplify the search for customers, google requires publishers to pick a category (from over 40 options) to publish their app under. However, with so many categories, many which are overlapping, publishers can have difficulty choosing the best category (Dogteiv, 2016). This project aims to help the publishers by building a bernoulli naive bayes model which recommends the most appropriate category for their app. 

By improving the categorisation of apps, customers will more easily locate apps of interest. This will benefit all parties involved. The customer finds the types of apps they are looking for, the publisher’s income increases and google receives a commission for the sale.






## B. Methods (max 1500 words)

### Pre-processing 

The datasets provided have a combined size of 641 megabytes and the notebook has a memory capacity of 12.78 gigabytes.  Without reducing the size of the data, analysis cannot be completed without reaching the memory capacity. In addition to this, the time it takes to train the model on the full dataset is excessive. In order to overcome these memory and time issues, a number of pre-processing methods have been used which are discussed below. 

##### Load data files

The files have been read in using pandas read.csv function. By default, pandas reads in the numbers as type 'float64'. In order to reduce the memory used to store the data, arguments are parsed to the function to read the numbers in as type 'float32'. This nearly halves the memory required.

Reading in the numbers as type 'float32' reduces the precision of the data. However, given that a bernoulli naïve bayes model has been used (discussed in the classifier section), the lose in precision will not impact the model or the results. 

In [0]:
# Create dictionaries for each column to pass into read.csv function
data_types_features = {0:'object'}
for i in np.arange(1,13628):
  data_types_features[i] = np.float32

data_types_desc = {0:'object', 1:'object'}
data_types_labels = {0:'object', 1:'object'}

In [11]:
print ("reading the dfs with explicit data types" , ctime())
# Read in as dataframes with explicit data types 
df_train = pd.read_csv('training_data.csv', header = None, dtype = data_types_features)
df_test =  pd.read_csv('test_data.csv', header = None, dtype = data_types_features)
df_desc =  pd.read_csv('training_desc.csv', header = None, dtype = data_types_desc)
df_labels =  pd.read_csv('training_labels.csv', header = None, dtype = data_types_labels)
print ("reading the dfs with explicit data types ends" , ctime())

datasets = [df_train, df_test, df_desc, df_labels]
names = ['training', 'test', 'description', 'labels']
i = 0
for data in datasets:
  print('The {} data has {} rows and {} columns'.format(names[i], data.shape[0], data.shape[1]))
  i += 1

('reading the dfs with explicit data types', 'Sun Jun  3 08:56:25 2018')
('reading the dfs with explicit data types ends', 'Sun Jun  3 08:57:37 2018')
The training data has 20104 rows and 13627 columns
The test data has 2233 rows and 13627 columns
The description data has 20104 rows and 2 columns
The labels data has 20104 rows and 2 columns


### Time = 66 seconds

Note, the following blocks of code involves preparing the data for further processing. It includes naming the columns, joining the train_df with labels_df and converting the labels to integers to facilitate the naïve bayes algorithm.

In [0]:
# rename columns -> makes finding the correct column after the join easier
df_train.columns = df_train.columns.values.astype(str)
df_train.rename(columns = {'0':'name'}, inplace = True)

df_test.columns = df_test.columns.values.astype(str)
df_test.rename(columns = {'0':'name'}, inplace = True)

df_desc.columns = df_desc.columns.values.astype(str)
df_desc.rename(columns = {'0':'name', '1':'description'}, inplace = True)

df_labels.columns = df_labels.columns.values.astype(str)
df_labels.rename(columns = {'0':'name', '1':'class'}, inplace = True)

In [0]:
# Convert class labels to integers
unique_classes = df_labels['class'].unique()

class_labels = {}
i = 0
for class_ in unique_classes:
  class_labels[class_] = i
  i += 1
  
# Map to df_labels
df_labels['class_int'] = df_labels['class'].map(class_labels)

In [0]:
# Join train_data with labels data
df_train = pd.merge(df_train, df_labels, how = 'inner', left_on = 'name', right_on = 'name')

# Drop class column -> So the type int label is only included
df_train.drop(['class'], axis = 1, inplace = True)

In [15]:
df_train.head()

,name,1,2,3,4,5,6,7,8,9,...,13618,13619,13620,13621,13622,13623,13624,13625,13626,class_int
0,com.borderstylo.retrollect,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,com.ogsm.respool,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,com.gasolin.android.avdhelper,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,com.droidhermes.birdjump,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
4,com.robotentertainment.heroacademy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


##### Filter observations

To reduce the time taken the train the model, observations have been filtered based on the size of their description. A threshold of 85 words was chosen as a descriptive app is generally considered to have at least 100 words (Olabenjo, 2016). This results in 6224 apps being removed from the training set.

This method was chosen over randomly sampling the data as the logic is that apps with small descriptions will create noise in the data. For example, an app with a small description will likely contain a feature vector with more zeros than ones compared to apps from the same class. However, had the description been longer, it may have more closely resembled other apps from its class. Therefore, small app description may be creating noise in the data and by removing them, the hope is it will increase the models performance.

In [16]:
# Count words in each description
word_count = []
desc_array = df_desc['description'].values
for desc in desc_array:
  words = len(desc.split(' '))
  word_count.append(words)

# Add word_count list to df_desc
df_desc['word_count'] = word_count

# Find apps which have 85 or less words, then filter training df
thresehold = 85
delete_apps = df_desc.loc[df_desc['word_count'] <= thresehold, 'name'].values.tolist()
df_train = df_train.loc[~(df_train['name'].isin(delete_apps)), :].reset_index(drop = True)

print('Number of observations removed: {}'.format(len(delete_apps)))

Number of observations removed: 6224


##### Train validation split

The df_train data has been split into a train set and validation set. The model will be trained on the train set and tested on the validation set. Despite cross validation considered a better method for validating a model (in terms of providing more reliable results), this method was chosen because training and testing a model multiple times was taking to long. 

Stratified sampling has been used to split the training data into a train set and validation set. This will ensure the training set is a fair representation of what the data would have looked like had it been trained on the full dataset.


In [17]:
# Stratified sampling from train
thresehold = int((0.6*(len(df_train)))) 

# Find the observations per class, then calculate the amount of samples which will be selected for each class
obs_per_class = df_train['class_int'].value_counts().reset_index()
obs_per_class.rename(columns = {'index':'class_int', 'class_int':'count'}, inplace = True)
obs_per_class['fraction'] = obs_per_class['count']/float(df_train.shape[0])
obs_per_class['val_sample'] = obs_per_class['fraction'] * thresehold
print(obs_per_class.head())

# Create a dictionary of the class label and the amount of samples to be included
sample_dict_thresehold = dict(zip(obs_per_class['class_int'], obs_per_class['val_sample']))

   class_int  count  fraction  val_sample
0         10    567  0.040850       340.2
1          0    555  0.039986       333.0
2          4    551  0.039697       330.6
3         15    543  0.039121       325.8
4          9    532  0.038329       319.2


In [18]:
# Create dictionary with all the classes and set to 0 for each
class_count = {}
classes = df_labels['class_int'].values.tolist()

for class_ in classes:
  class_count[class_] = 0

counter = 0
row_idx = []  
# Shuffle dataframe -> importantly, the index value stays the same
np.random.seed(1)
df_train = df_train.iloc[np.random.permutation(len(df_train))]

class_col = df_train.columns.values.tolist().index('class_int')
# Loop though dataframe appending i value of row when that rows class count is less than its thesehold
i = 0
while i < len(df_train):
  class_ = df_train.iat[i, class_col]
  if class_count[class_] < sample_dict_thresehold[class_]:
    row_idx.append(i)
    counter += 1
    class_count[class_] += 1
    
    # Break once all the samples have been collected
    if counter == thresehold:
      break
    
  i += 1

# find idx value of sampled rows
keep_idx = df_train.index.values[row_idx]

# Filter df_train
df_validation = df_train.iloc[df_train.index.isin(keep_idx), :]
df_train = df_train.iloc[~(df_train.index.isin(keep_idx)), :]

print('Training data has {} observations'.format(df_train.shape[0]))
print('Validation data has {} observations'.format(df_validation.shape[0]))

Training data has 5552 observations
Validation data has 8328 observations


The general rule of thumb is to allocated more data to the training set - generally around 80% to train and 20% to validation. However, to increase the speed of the model training, the data has been split 40% train and 60% validation. It is likely this increase in training speed will come at the cost of a reduction in accuracy due to model parameters being trained on less data

Note, the following code blocks relate to converting the features to binary. More explanation on why will be provided in the classifier section.

In [0]:
# Convert dataframes into arrays for NB algorithm. The first column is the apps name and hence has been cut from all the arrays
df_train_array = df_train.iloc[:, 1:].values
df_validation_array = df_validation.iloc[:, 1:].values
df_test_array = df_test.iloc[:, 1:].values

In [0]:
# Function which converts features into binary
def convert_to_boolean(x):
    if x > 0.0:
        return True
    else:
        return False
    
conversion = np.vectorize(convert_to_boolean)

In [16]:
print ("Converting the feature set begins" ,ctime())
# Apply the function above to convert the features
arrays = [df_train_array, df_validation_array]
for array in arrays:
  for i in range(df_train_array.shape[1] - 1): # Exclude label column
    # Convert array to binary
    converted_feature = conversion(array[:,i])
    array[:, i] = converted_feature


for i in range(df_test_array.shape[1]): # no label
  converted_feature = conversion(df_test_array[:,i])
  df_test_array[:, i] = converted_feature
  
print ("Converting the feature set ends" ,ctime())

('Converting the feature set begins', 'Mon May  7 00:11:31 2018')
('Converting the feature set ends', 'Mon May  7 00:12:03 2018')


### Time = 30 seconds

---

##### Feature selection

Feature selection has been implemented to increase the speed of the algorithm. Feature selection is often used in text classification due to the high dimensionality of the data (Schneider, 2005).  In addition to this, text classification problems have a lot of the features which are noisy and redundant (Narayanan et al, 2013). By removing such features, the model’s performance can improve. Ideally feature selection would be used in the train and testing stage of the process to find the set of features which has the best out of sample accuracy. However, it has been implemented here to increase the speed of the training. 

There are a handful of feature selection methods which are commonly applied to text classification with naïve bayes models, all of which use a greedy approach (Schneider, 2005). This project applied a mutual information approach. This approach provides a theoretic measure of the information a feature gives about the class (Schneider, 2005). The formula is shown below: 


$I(X:Y) = \Sigma_{y \epsilon Y}\Sigma_{x \epsilon X} p(x,y)log \frac{p(x,y)}{p(x)p(y)}$

Where:
* X is the specific feature
* Y is the response variable
* p(x,y) is the joint probability function of X and Y
* p(X) and p(y) are marginal probility functions of X and Y

For the bernoulli model, X will be either 1 or 0 and Y will range between 0 and 29 (as a result of the data having 30 classes)

The formula returns a non-negative value. Mutual information is equal zero when the X and Y variables are independent and the more dependent, the higher the value. Additionally, when joint probability is zero, the algorithm sets the value of that specific dependency to zero instead of minus infinity.

Note, the mutual information values for each column have been calculated on the training set, not the combined train and validation sets.

In [0]:
# Hold the mutual information values for each column
MI_value = np.zeros((df_train_array[:,:-1].shape[1],1))

# # Find the marginal probability of each class
y = df_train_array[:, -1]
n_rows = df_train_array.shape[0]
unique_class = np.unique(y)
prob_class = {}
for class_ in unique_class:
    variable_name = 'class_' + str(class_)
    class_count = len(y[np.where(y == class_)])
    prob_class[variable_name] = class_count/float(n_rows)

# For each column, find the marginal probability of 1, 0 and store in dictionary
conditional_probs = {}
joint_probs = {}
marginal_probs = {}
for i in range(df_train_array.shape[1] - 1): #exclude label column
    X = df_train_array[:, i]  
    marg_prob_col_1 = 'prob_1_col_{}'.format(i)
    marg_prob_col_2 = 'prob_0_col_{}'.format(i)
    prob_word = len(X[np.where(X == 1.0)])/float(n_rows)
    marginal_probs[marg_prob_col_1] = prob_word
    marginal_probs[marg_prob_col_2] = 1 - prob_word
    
    # For each class and on the specific column, calculate the conditional probability of 1 or 0
    for j in unique_class:        
        variable_name_1 = '1_given_{}_for_col_{}'.format(j, i)
        variable_name_2 = '0_given_{}_for_col_{}'.format(j, i)
        X_filtered_by_class = np.where(y == j)
        X_equals_1 = np.sum(X[X_filtered_by_class])/len(X[X_filtered_by_class])
        conditional_probs[variable_name_1] = X_equals_1
        conditional_probs[variable_name_2] = 1 - X_equals_1
        # Use Bayes theorem to find joint probabilities
        joint_name_1 = '1_and_{}_col_{}'.format(j, i)
        joint_name_2 = '0_and_{}_col_{}'.format(j, i)
        joint_probs[joint_name_1] = conditional_probs[variable_name_1] * prob_class['class_' + str(j)]
        joint_probs[joint_name_2] = conditional_probs[variable_name_2] * prob_class['class_' + str(j)]

In [0]:
for i in range(df_train_array.shape[1] - 1): # Exclude label
    MI = 0
    # iteratively update MI for each class word/no word calculation 
    for j in unique_class:
        for k in [0,1]: # Word or no word
            joint = joint_probs['{}_and_{}_col_{}'.format(k,j,i)]
            if joint == 0.0: # Set MI value to zero for specific class and word no word calc for the column
                continue
            
            MI += joint * np.log2(joint/(marginal_probs['prob_{}_col_{}'.format(k,i)] *
                                         prob_class['class_{}'.format(j)]))
    MI_value[i] = MI


After trying a range of different feature sizes, it was found that the largest subset of columns to train the model with which ran in a reasonable time was 10%.

In [0]:
# Create array with the first column holding the columns and the second column the correspoding MI value
columns = np.arange(0.0,MI_value.shape[0],1).reshape(MI_value.shape[0],1)
MI_values = np.concatenate((columns, MI_value), axis = 1)

# Order array from higher value to lower
MI_values = MI_values[MI_values[:,1].argsort()]
MI_values = np.flipud(MI_values)


In [0]:
# Create y arrays from train and validation sets
y_train = df_train_array[:,-1].reshape(-1,1)
y_val = df_validation_array[:,-1].reshape(-1,1)

# Filter all the arrays to include only the top 10% of features based on mutual information scores
columns = int(0.1 * MI_values.shape[0])
filtered_columns_idx = MI_values[0:columns, 0].astype(int)

df_train_array = df_train_array[:, filtered_columns_idx]
df_train_array = np.concatenate((df_train_array, y_train), axis = 1)
df_validation_array = df_validation_array[:, filtered_columns_idx]
df_validation_array = np.concatenate((df_validation_array, y_val), axis = 1)
df_test_array = df_test_array[:, filtered_columns_idx]

### Classifier

This project implements a naïve bayes classifier. The naïve bayes model uses a probabilistic framework where it predicts the most likely class by using bayes rule (Schneider, 2005). The model is fast and simple and as a result, has been widely used in text classification problems (Narayanan et al, 2013). At the expense of accuracy, the naïve bayes model assumes independence between words in the document. Without this assumption, the training would be too computationally intensive (Schneider, 2005).

There are a range of naïve bayes models available, differing on the modelling of words in the document (Schneider, 2005). The three most common are: Bernoulli, multinomial and poisson. The Bernoulli model is a binary model, 1 of the word exists, 0 if it does not. The latter two models require the count of each word in the document (Schneider, 2005). As the data provided uses tf-idf values and there was no description on how these values were calculated (using frequency etc), the multinomial and poisson models were not applicable. Hence, the bernoulli model has been applied. To convert the data into binary values, we have assumes values greater than 0 represent the document having the word.


The Navies Bayes Classifier works by calculating the posterior probability P(c|X) from the prior probability of class P(c), the likelihood of the class P(X|c) and the prior probability of predictor P(X). Bayes theorem can be represented by the equation below:




$P(c|X) = \frac{P(X|c)P(c)}{P(X)} = \frac{P(c)}{P(X)} \Pi_{n=1}^{t}P(X{n}|c)$
 

Where t is the total no of attributes and X${n}$ is the value of i$^{th}$ attribute

As there are 30 classes in the dataset we will be classifying apps based on the highest probability for each category. Hence the equation for our classification will be maximised on the basis of each category. The equation for the same is as shown below:

$Class(App) = max(P(c)\Pi_{n=1}^{t}P(X{n}|c)$

However, using the equation above can result in floating point underflow as a result of all the conditional probabilities being multiplied together. To overcome this, the equation is modified by adding logarithms of the probabilities instead of multiplying. This is shown in the equation below (Christoper, 2009).

$Class(App) = max(log(P(c)) + \Sigma_{n=1}^{t}log(P(X{n}|c)$

Laplace smoothing is used to avoid the situation wherein the model assigns a probability of zero as it has not seen the event in the training data.
The probability P(C) and P(X|c) can be corrected as follows:

$P(X{i}|c) = \frac{|D{cxi} + 1|}{|Dc| + Ni} $

---




### Training
Preprocessed df_train was used in training our classifier. In the training phase we are calculating prior probabilities for each category and conditional probability for each column. 
We start the training phase by taking in the feature array and counting the number of features.
We initialize two dictionaries that will store the prior probability and conditional probability of each class.
We then loop each row and before we proceed to calculate the prior and the conditional probabilities we check to only include those rows which include the class. We then proceed to calculate the prior probability of the class and store it in the pre-initialized dictionary. 

We then proceed to find the conditional probabilites for each column in the row. We loop through the features counting the number of rows which have a value for the specific class. We then sum the conditional probabilites for each column in the row and then add 1 as part of the Laplace smoothening.After the adding all the conditional probabilites we then proceed to complete the second part of smoothening where in we divide the conditional probabiility by the sum of the number of features for that class and 2.
We then complete one cycle on training phase by storing the conditional probability in the dictionary.

In [21]:
print ("training starts" , ctime())
### Train ###
# unique_class -> variable created earlier
features = df_train_array[:, :-1] # feature array
n_features = features.shape[1] # number of features
# n_rows = features.shape[0] -> created earlier

# Create empty dictionaries which will store the priors and conditional probs
priors_dict = {}
conditional_dict = {}

# Loop through each class to calculate prior and conditional probs
for class_ in classes:
    # filter array to only rows which include the class
    class_rows = df_train_array[np.where(y == class_)] # y variable was created earlier
    # find prior
    n_rows_class = class_rows.shape[0] # number of rows with the specific class
    prior = n_rows_class/(float(n_rows)) # prior
    # Store prior in dictionary
    variable_name = 'class_' + str(class_)
    priors_dict[variable_name] = prior
    
    #Find column conditional probabilities 
    class_cond_prob = np.zeros((n_features,1)) # vector of zeros which will be updated to have each columns cond prob for the class
    # Loop through the features counting the number of rows which have a value for the specific class
    for i in range(n_features):
        feature = class_rows[:, i] 
        class_feature_sum = np.sum(feature) # sum feature
        class_cond_prob[i] = class_feature_sum + 1 # update the cond prob vector for each feature - 1 is part of laplace smoothng
    # Convert 
    class_cond_prob = class_cond_prob/float((n_rows_class + 2)) # 2 is the second part of the smoothing
    # store conditional probs for the class in dictionary
    conditional_dict[variable_name] = class_cond_prob
    
print ("training ends" ,ctime())

('training starts', 'Mon May  7 00:12:43 2018')
('training ends', 'Mon May  7 00:15:47 2018')


### Time = ~209 seconds

### Testing

In the testing phase we taken in the dataset over which we need to test out the results (Validation and Actual Testing). We make use of the dictionaries for prior and conditional probabilities created earlier to classify the testing data.

We start by initializing an array to store the prediction. We then loop for each row of the test data to calcualte prior probability and conditional probability of each app and column respectively for each class. We then proceed to calculate the final probability of each app by adding the logarithmic prior probability and conditional probabability for each class. We finally store the maximum probability for the app after calculating all the probabilites and based on the maxiumum probability we assign our test app the class of the highest probability

In [1]:
### Predict ###
def predict(array):
  predictions = np.zeros((array.shape[0],1))
  for i in range(array.shape[0]):
    new_row = array[i, :-1]
    posteriors = np.zeros((len(unique_class),1))
    for j in range(posteriors.shape[0]):     
      variable = 'class_' + str(j)
      prior_prob = priors_dict[variable]
      cond_prob_vector = conditional_dict[variable].flatten() 
      prob = np.log(prior_prob) + np.log(np.prod(cond_prob_vector[np.where(new_row == True)])) + \
                                  np.log(np.prod(1-cond_prob_vector[np.where(new_row ==False)]))
      posteriors[j] = prob

    predictions[i] = np.argmax(posteriors)
    
  return predictions.ravel()

In [23]:
print ("Validation Starts" ,ctime())
preds = predict(df_validation_array)
print ("Validation ends" ,ctime())

('Validation Starts', 'Mon May  7 00:16:03 2018')
('Validation ends', 'Mon May  7 00:16:18 2018')


### Time = ~15 seconds

In [0]:
y = y_val.reshape(y_val.shape[0], 1)
preds = preds.reshape(preds.shape[0],1)

In [0]:
results = np.concatenate((preds, y), axis = 1)
results = results.astype(int)

## C. Experiments and Results (max 500 words)

### Accuracy

The accuracy of the classfier for the for validation set is 58.47%. This is a significant improvement over a naive model which always assumes the most popular label. 

In [26]:
np.round((np.sum(preds == y)/round(preds.shape[0]))*100,2)

58.47

In [27]:
# getting the shape of confusion matrix from the number of classes
uniqueClasses = np.unique(results)
conf_matrix = np.zeros(shape=(uniqueClasses.size, uniqueClasses.size))

# creating an emptry matrix for the confusion matrix
rows = results.shape[0]
for x in range(rows):
  actual = results[x][0]
  predicted = results[x][1]
  conf_matrix[actual][predicted] += 1
  
# installing a library to better visualize the data
!pip install -U -q tabulate
from tabulate import tabulate

# get headers
table_labels = df_labels[['class']].drop_duplicates().values.tolist()
headers = table_labels

# tabulate data
table = tabulate(conf_matrix, headers, showindex=headers, tablefmt="fancy_grid")

# output
print(table)

╒═════════════════════════╤═══════════════════╤══════════════╤══════════════════════════╤═════════════════════════╤══════════════════════════╤═════════════════════╤════════════════╤═══════════════════════╤═════════════╤════════════════════════╤═════════════════╤══════════════════════════╤═══════════════════════╤══════════════╤═══════════════╤══════════════╤════════════════════════╤═════════════════════╤════════════════════╤═══════════════════════════╤══════════════╤═══════════════════════╤══════════════╤════════════════════════╤════════════════╤══════════════════════╤═══════════════╤═════════════════╤═══════════════╤════════════════════╕
│                         │   ['Photography'] │   ['Social'] │   ['Libraries and Demo'] │   ['Arcade and Action'] │   ['Health and Fitness'] │   ['Communication'] │   ['Shopping'] │   ['Music and Audio'] │   ['Tools'] │   ['Brain and Puzzle'] │   ['Education'] │   ['News and Magazines'] │   ['Personalization'] │   ['Racing'] │   ['Medical'] │   ['Casua

In [28]:
TP_TN_FP_FN_ = np.zeros(shape=(conf_matrix.shape[0], 8)) # extra 4 cols for precision, recall, acc and f score

TP = conf_matrix.diagonal()
TP_TN_FP_FN_[:,0] = TP

sum_ = conf_matrix.sum() 
for x in range(TP_TN_FP_FN_.shape[0]):
  TP_TN_FP_FN_[x][2] = conf_matrix[:,x].sum() - TP_TN_FP_FN_[x][0] #false positive
  TP_TN_FP_FN_[x][3] = conf_matrix[x].sum() - TP_TN_FP_FN_[x][0] #false negative

for x in range(TP_TN_FP_FN_.shape[0]):
  TP_TN_FP_FN_[x][1] = sum_ - TP_TN_FP_FN_[x][0] - TP_TN_FP_FN_[x][2] - TP_TN_FP_FN_[x][3] #true negative
  
# precision tp/(tp+fp)
for x in range(TP_TN_FP_FN_.shape[0]):
  TP_TN_FP_FN_[x][4] = TP_TN_FP_FN_[x][0] / (TP_TN_FP_FN_[x][0] + TP_TN_FP_FN_[x][2])

# recall tp/(tp+fn)
for x in range(TP_TN_FP_FN_.shape[0]):
  TP_TN_FP_FN_[x][5] = TP_TN_FP_FN_[x][0] / (TP_TN_FP_FN_[x][0] + TP_TN_FP_FN_[x][3])

# accuracy = (tp+tn)/(tp+fn+fp+tn)
for x in range(TP_TN_FP_FN_.shape[0]):
  TP_TN_FP_FN_[x][6] = (TP_TN_FP_FN_[x][0] + TP_TN_FP_FN_[x][1]) / sum_

# f_measure = 2 * tp / (2 * tp + fn + fp) or 2 / ((1/precision) + (1/recall))
for x in range(TP_TN_FP_FN_.shape[0]):
#   TP_TN_FP_FN_[x][7] = 2 * TP_TN_FP_FN_[x][0] / (2 * TP_TN_FP_FN_[x][0] + TP_TN_FP_FN_[x][3] + TP_TN_FP_FN_[x][2]) #same
  TP_TN_FP_FN_[x][7] = 2 / ((1/TP_TN_FP_FN_[x][4]) + (1/TP_TN_FP_FN_[x][5]))

  
headers = ["TP", "TN", "FP", "FN", "Precision", "Recall", "Accuracy", "F-score"]
truthtable = tabulate(TP_TN_FP_FN_, headers, showindex=table_labels, tablefmt="fancy_grid")

# output
print(truthtable)
# print("TP_TN_FP_FN_\n", TP_TN_FP_FN_)

╒═════════════════════════╤══════╤══════╤══════╤══════╤═════════════╤══════════╤════════════╤═══════════╕
│                         │   TP │   TN │   FP │   FN │   Precision │   Recall │   Accuracy │   F-score │
╞═════════════════════════╪══════╪══════╪══════╪══════╪═════════════╪══════════╪════════════╪═══════════╡
│ ['Photography']         │  246 │ 7855 │   87 │  140 │    0.738739 │ 0.637306 │   0.972743 │  0.684284 │
├─────────────────────────┼──────┼──────┼──────┼──────┼─────────────┼──────────┼────────────┼───────────┤
│ ['Social']              │  165 │ 7900 │  114 │  149 │    0.591398 │ 0.525478 │   0.96842  │  0.556492 │
├─────────────────────────┼──────┼──────┼──────┼──────┼─────────────┼──────────┼────────────┼───────────┤
│ ['Libraries and Demo']  │    9 │ 8242 │   71 │    6 │    0.1125   │ 0.6      │   0.990754 │  0.189474 │
├─────────────────────────┼──────┼──────┼──────┼──────┼─────────────┼──────────┼────────────┼───────────┤
│ ['Arcade and Action']   │  206 │ 7873 │  109

### Extensive Analysis



From the confusion matrix charted below(actual class on y-axis, prediction on x-axis), it can be observed the classifier is generally able to catch the trend of the data. However, we can there are some classes where the classifier is able to predict more accurately than the other comparing their respective f-score. Classes like 'News and Magazines', 'Racing', 'Cards and Casino' are some of the classes where the classifier maintains more than 0.70 f-score. The observation that can be made here on this context is that these classes have unique keywords that are not used in other class of applications. 'Cards and Casino' contain words like 'cards', specific card board game names which helps the classifier to maintain the higher score.

Then we can also observe, some classes where the classifier makes poor predictions. 'Libraries and Demo' is one of the classes where the precision is very low (0.11). This specific class in question has the lowest number of support(instances) to train the classifier. 'Lifestyle' is an interesting category where despite having number of support instances, the classfier often identified the class as 'Social', 'Health and Fitness', 'Shopping', 'Entertainment', 'Books and References' and myriad other other classes. This however isn't as surprising considering 'Lifestyle' does not contain any strong unique identifiers that seperates it from other classes and often a overlap of application of different classes or genres. This class has a f-score of 0.18.

Another interesting observation can be made for 'Books and References' class. This particular class was often misclassifed as 'Education'. The class 'Education' was also frequently misclassfied as 'Books and References', and 'Brain and Puzzle'. A case can be made for how similar this classes are, and what really sepearates between classes like 'Education' and 'Books and References'. The class 'comics' is interesting for its higher recall(0.7) and lower precision(0.21). This class 'Comics' in notable number of cases was predicted as 'Personalisation' & 'Entertainment'.

A detailed table is available below which shows the True Positive, True Negative, False Positive, False Negatives for each of the classes. This numbers were used to find the precision, recall, accuracy and f-score for each of the classes. The accuracy for each class hovers around 95% or more, however this result was grossly amplified by the classfier simply not predicting that class for the instances. This weakness of accuracy is why f-measure is used which takes precision and recall into account to give an overall score.

## D. Export Results 

This is important for your grading.


You must save a file named “**predicted_labels.csv**” in the **same data format** as “training_labels.csv”.

You can use PyDrive to save the data file (example is not provided here and you should find out how to do it on your own).

Make sure the predictions (classiﬁcation results for the test set) are in the **same order** as test inputs, i.e. the ﬁrst row of “predicted_labels.csv” corresponds to the ﬁrst row of “test_data.csv” and so on). 

Your score will be based on how accurate your approach is. We will collect “predicted_labels.csv” and compare it to the actual labels to get the accuracy of your approach. For further testing purposes, we may use a diﬀerent test set while grading.


In [38]:
print ("predictions start" ,ctime())
# Make predictions
test_predictions = predict(df_test_array).astype(int)
# Append predictions to df_test
df_test['int_preds'] = test_predictions
# Invert the dictionary created at the start to map labels to integers
inv_labels_dict = {v: k for k, v in class_labels.items()}
# Map int predictions to labels
df_test['label'] = df_test['int_preds'].map(inv_labels_dict)
df_final = df_test.loc[:, ['name', 'label']]
print ("predictions end" ,ctime())
header = ["name", "label"]
df_final.to_csv("test.csv",columns=header,index=False , sep=';',header=False)

('predictions start', 'Mon May  7 00:47:56 2018')
('predictions end', 'Mon May  7 00:48:00 2018')


### Time = ~5 seconds

In [0]:
gfile = drive.CreateFile({'title':'predicted_labels.csv', 'mimeType':'text/csv',
        "parents": [{"id":'13AVnd-fGVAcIIJZy6ParOFaJa_xyr5YX' }]})
gfile.SetContentFile("test.csv")
gfile.Upload()

**Insert the url address of your predicted_labels.csv file:   **



## E. Discussion (max 300 words)

As discussed in the extensive analysis section, there are a handful of classes which the model struggles to differentiate between due to there similarity. What could create differences between these classes is a multinomial model rather than bernoulli. The multinomial model considers the frequency of the word rather than just if it is present or not. This would help seperate classes which have overlapping charisteristics.

With some classes being very similar, what is considered the correct label for an app? For this project, we have assumed the data provided has the truth as its labels. However, this is a subjective decision where in most cases an app could reasonably belong to multiple classes. In the example given above, it is possible that two publishes could be given the same app and one choose ‘Transportation’ and the other ‘Travel & Local’.  This creates noise in the response variable and reduces the models ability to accurately make predictions. 

In addition to a noisy response variable, the feature selection section briefly touched on text classification problems having noisy features. When this is present, the accuracy of the model can be negatively affected (Narayanan et al, 2013). To find the features which maximise accuracy, the model could have been trained and tested multiple times using subsets of the features based on there mutual information values. However, this process was not implemented due to the computational intensity.

## F. Future work

Further improvements can be made to this classification task through the following changes:
* Use a multinomial naïve bayes model. This model takes into account the count of the words in the document. It is likely this would improve the accuracy of the model, especially for the classes which have overlapping words.
* Using more powerful algorithms such as support vector machines. 
* If the we were to stay with the Bernoulli model, increasing the efficiency of the algorithm would allow for model tuning through feature selection. Note, cross validation would be used to select the best model. As the current train/validation split method can be susceptible to high bias. Additionally, a more efficient algorithm would allow for the model to be trained on more data, hence, likely improving the accuracy.


## G. Conclusion

In this assignment, we were able to learn the complexities involved with handling sparse data  in a classification task with limited memory. While the implemented Naive Bayes was able to offer a respectable result, through our analysis we have offered examples of how classes often get misclassified by having descriptions in similar context and not providing strong unique identifiers. This accuracy of the model could be improved by increases the speed of the algorithm which would allow for a more rigourous feature selection experiemnt. Additionally, classifiers such as the multinomial model or support vector machines could increase the accuracy.

## H. References

Christopher D. Manning, C., 'Introduction to information retrieval', chapter 13, Cambridge University Press 

Dogtiev, A., 2016, ‘How to choose the right app store category’, Business of Apps, http://www.businessofapps.com/how-to-choose-the-right-app-store-category/

Narayanan, V., Arora, I., Bhatia, A., 2013, ‘Fast and accurate sentiment classification using an enhanced Naive Bayes model’, Indian Institute of Technology

Olabenjo, B., 2016, ‘Applying naïve bayes apps classification to google play apps categorization’, University of Saskatchewan

Schoon, B., 2016,  ‘Google Play Store picks up 8 new app categories to bolster discovery experience’ 9TO5Google, discovery experience https://9to5google.com/2016/07/27/google-play-store-new-categories/

Unknown, 2013, ‘Thousands of health apps but so few downloads?’, The Sun Daily, http://www.thesundaily.my/news/871252